模型导入

In [1]:
import os
import torch
from tensorflow import keras
from torch import cuda,nn
from NN_class import LeNet
import numpy as np

device = 'cuda' if cuda.is_available() else 'cpu'

loaded_params = torch.load('save_model\\best_model_on_3.pth')
net = LeNet().to(device)
net.load_state_dict(loaded_params)

<All keys matched successfully>

定义特征图展示函数

In [2]:
import time
import matplotlib.pyplot as plt

def draw_features(width,height,x,savename):
    tic=time.time()
    fig = plt.figure(figsize=(12,16))
    fig.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=0.95, wspace=0.05, hspace=0.05)
    for i in range(width*height):
        plt.subplot(height,width, i + 1)
        plt.axis('off')
        # plt.tight_layout()
        img = x[i, :, :]
        pmin = np.min(img)
        pmax = np.max(img)
        img = (img - pmin) / (pmax - pmin + 0.0001)
        img = img.T
        plt.imshow(img, cmap='viridis')
        print("{}/{}".format(i,width*height))
    fig.savefig(savename, dpi=1000)
    fig.show()
    fig.clf()
    plt.close()
    print("time:{}".format(time.time()-tic))



图片读入

In [3]:
from keras.preprocessing import image
import numpy as np
from torchvision import transforms

# The local path to our target image
data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0,0,0], std=[1.0,1.0,1.0])
])

# Prepare image
img_path = "test_3.jpg"
assert os.path.exists(img_path), "file: '{}' dose not exist.".format(img_path)
img = image.load_img(img_path, target_size=(28, 28))
img = np.array(img, dtype=np.float32)
img_tensor = data_transform(img)
img_tensor = img_tensor.permute(0,2,1)



# `img` is a PIL image of size 224x224

x = img_tensor.to(device)



结果输出

In [4]:
output = net(x)
y_pred = torch.argmax(output)#求最大的类别的索引
print(y_pred)

tensor(3, device='cuda:0')


进行特征图展示

In [5]:
import tensorflow as tf
from tensorflow.keras import backend as K 



x1 = net.conv1(x)
draw_features(3, 2, x1.cpu().detach().numpy(), "{}/f1_conv1_viridis.png".format('plots'))  
print("{}/f1_conv1.png".format('plots'))

x2 = net.active_fun(x1)
draw_features(3, 2, x2.cpu().detach().numpy(), "{}/f1_act1_viridis.png".format('plots'))  
print("{}/f1_conv1.png".format('plots'))

x2 = net.pooling1(x2)
draw_features(3, 2, x2.cpu().detach().numpy(), "{}/f1_pooling1_viridis.png".format('plots'))  
print("{}/f1_conv1.png".format('plots'))

x2 = net.conv2(x2)
draw_features(4, 4, x2.cpu().detach().numpy(), "{}/f1_conv2_viridis.png".format('plots'))  
print("{}/f1_conv1.png".format('plots'))

x3 = net.active_fun(x2)
draw_features(4, 4, x3.cpu().detach().numpy(), "{}/f1_act2_viridis.png".format('plots'))  
print("{}/f1_conv1.png".format('plots'))

x3 = net.pooling2(x3)
draw_features(4, 4, x3.cpu().detach().numpy(), "{}/f1_pooling2_viridis.png".format('plots'))  
print("{}/f1_conv1.png".format('plots'))



0/6
1/6
2/6
3/6
4/6
5/6


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27536\3897836850.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


time:5.198038816452026
plots/f1_conv1.png
0/6
1/6
2/6
3/6
4/6
5/6
time:5.230624675750732
plots/f1_conv1.png
0/6
1/6
2/6
3/6
4/6
5/6
time:5.251984119415283
plots/f1_conv1.png
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
time:6.450160264968872
plots/f1_conv1.png
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
time:7.238378524780273
plots/f1_conv1.png
0/16
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
time:6.4469873905181885
plots/f1_conv1.png


卷积核可视化

In [6]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils
import torchvision.models as models

kernel_num = 0
vis_max = 2
# 取前两层卷积核
for sub_module in net.modules():
    if not isinstance(sub_module, nn.Conv2d):
        continue
    if kernel_num >= vis_max:
        break
    kernel_num += 1
    kernels = sub_module.weight
    print(kernels.shape)
    c_out, c_in, k_h, k_w = tuple(kernels.shape)  # 输出通道数,输入通道数,卷积核宽,卷积核高
    kernels = kernels.view(-1,k_h, k_w)
    print(kernels.shape)
    draw_features(c_in,c_out,kernels.cpu().detach().numpy(),'plots/kernel{}'.format(kernel_num))

    

        

torch.Size([6, 3, 5, 5])
torch.Size([18, 5, 5])
0/18
1/18
2/18
3/18
4/18
5/18
6/18
7/18
8/18
9/18
10/18
11/18
12/18
13/18
14/18
15/18
16/18
17/18


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27536\3897836850.py:20: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  fig.show()


time:5.854251861572266
torch.Size([16, 6, 5, 5])
torch.Size([96, 5, 5])
0/96
1/96
2/96
3/96
4/96
5/96
6/96
7/96
8/96
9/96
10/96
11/96
12/96
13/96
14/96
15/96
16/96
17/96
18/96
19/96
20/96
21/96
22/96
23/96
24/96
25/96
26/96
27/96
28/96
29/96
30/96
31/96
32/96
33/96
34/96
35/96
36/96
37/96
38/96
39/96
40/96
41/96
42/96
43/96
44/96
45/96
46/96
47/96
48/96
49/96
50/96
51/96
52/96
53/96
54/96
55/96
56/96
57/96
58/96
59/96
60/96
61/96
62/96
63/96
64/96
65/96
66/96
67/96
68/96
69/96
70/96
71/96
72/96
73/96
74/96
75/96
76/96
77/96
78/96
79/96
80/96
81/96
82/96
83/96
84/96
85/96
86/96
87/96
88/96
89/96
90/96
91/96
92/96
93/96
94/96
95/96
time:6.198883056640625


权重获取

In [10]:
import cv2
a = loaded_params['output.weight']
b = loaded_params['fully_connect.weight']
c = loaded_params['conv3.weight']
print(a.shape)
print(b.shape)
print(c.shape)


KeyError: 'fully_connect.weight'